In [1]:
import pandas as pd
import numpy as np

In [2]:
eggs = pd.read_csv("base_data.csv")

In [3]:
eggs.head()

,fisc_wk_id,fisc_wk_strt_dt,mkt_lvl,rgn_shrt_nm,co_nbr,co_shrt_nm,bus_ctr_nm,itm_grp_nm,attr_grp_nm,prc_src_typ_cd,cust_nbr,cuisine_lvl_1_desc,ttl_cases,ttl_sales,ttl_lines,ttl_grs_prft
0,202249,2022-06-05,South,South Texas,67,HOUSTON,DAIRY,EGGS,SHELL EGGS,CEP,1628,Asian,2.0,67.76,1,13.560
1,202249,2022-06-05,South,South Texas,67,HOUSTON,DAIRY,EGGS,SHELL EGGS,CEP,3129,Latin,2.0,75.90,1,15.704
2,202249,2022-06-05,South,South Texas,67,HOUSTON,DAIRY,EGGS,SHELL EGGS,CEP,10470,Latin,3.0,113.67,1,23.377
3,202249,2022-06-05,South,South Texas,67,HOUSTON,DAIRY,EGGS,SHELL EGGS,CEP,11759,Asian,2.0,77.58,1,17.384
4,202249,2022-06-05,South,South Texas,67,HOUSTON,DAIRY,EGGS,SHELL EGGS,CEP,11846,NaN,5.0,194.90,1,44.412


In [4]:
customers = eggs["cust_nbr"].unique()

In [5]:
# databefore 202340

e_39 = eggs[eggs["fisc_wk_id"] < 202340]

In [6]:
wk_39 = e_39["fisc_wk_id"].unique()

In [7]:
wk_39

array([202249, 202250, 202251, 202252, 202301, 202302, 202303, 202304,
       202305, 202306, 202307, 202308, 202309, 202310, 202311, 202312,
       202313, 202314, 202315, 202316, 202317, 202318, 202319, 202320,
       202321, 202322, 202323, 202324, 202325, 202326, 202327, 202328,
       202329, 202330, 202331, 202332, 202333, 202334, 202335, 202336,
       202337, 202338, 202339], dtype=int64)

In [8]:
full = []
for i in customers:
    full.append([i]*wk_39.size)

In [9]:
np.array(full).flatten()

array([  1628,   1628,   1628, ..., 945998, 945998, 945998], dtype=int64)

In [10]:
len(wk_39.tolist()*customers.size)

35088

In [11]:
df = pd.DataFrame(wk_39.tolist()*customers.size, columns = ["fisc_wk"])

In [12]:
df["cust"] = np.array(full).flatten()

In [13]:
df

,fisc_wk,cust
0,202249,1628
1,202250,1628
2,202251,1628
3,202252,1628
4,202301,1628
...,...,...
35083,202335,945998
35084,202336,945998
35085,202337,945998
35086,202338,945998


In [14]:
right = e_39[["fisc_wk_id","cust_nbr"]]

In [15]:
final = df.merge(right, left_on=["fisc_wk","cust"], right_on = ["fisc_wk_id","cust_nbr"], how = "left")

In [16]:
final["Purchases"] = 0

In [17]:
final.loc[final["cust_nbr"].notnull(),"Purchases"] = 1

In [18]:
final

,fisc_wk,cust,fisc_wk_id,cust_nbr,Purchases
0,202249,1628,202249.0,1628.0,1
1,202250,1628,202250.0,1628.0,1
2,202251,1628,202251.0,1628.0,1
3,202252,1628,NaN,NaN,0
4,202301,1628,NaN,NaN,0
...,...,...,...,...,...
35083,202335,945998,NaN,NaN,0
35084,202336,945998,NaN,NaN,0
35085,202337,945998,NaN,NaN,0
35086,202338,945998,NaN,NaN,0


In [19]:
final.columns

Index(['fisc_wk', 'cust', 'fisc_wk_id', 'cust_nbr', 'Purchases'], dtype='object')

In [20]:
final = final[['fisc_wk', 'cust', 'Purchases']].sort_values("fisc_wk").reset_index(drop= True)

In [21]:
final.columns = ["fisc_wk_id","cust_nbr",'Purchases']

In [22]:
final

,fisc_wk_id,cust_nbr,Purchases
0,202249,1628,1
1,202249,732420,0
2,202249,925347,0
3,202249,733840,0
4,202249,917344,0
...,...,...,...
35083,202339,25818,0
35084,202339,23485,0
35085,202339,13484,0
35086,202339,963249,0


In [23]:
final.to_csv("purchasing_202339.csv", index = False)

In [24]:
eggs.shape

(6135, 16)

In [25]:
final.shape

(35088, 3)

In [26]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35088 entries, 0 to 35087
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   fisc_wk_id  35088 non-null  int64
 1   cust_nbr    35088 non-null  int64
 2   Purchases   35088 non-null  int64
dtypes: int64(3)
memory usage: 822.5 KB


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

In [28]:
final['year'] = final['fisc_wk_id'].astype(str).str[:4].astype(int)
final['week'] = final['fisc_wk_id'].astype(str).str[4:].astype(int)

In [29]:
X = final[['year', 'week', 'cust_nbr']]
y = final['Purchases']

In [30]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Fit the model on the training data
# rf_classifier.fit(X_train, y_train)
rf_classifier.fit(X, y)


RandomForestClassifier()

In [72]:
# Initialize the RandomForestClassifier
lr_classifier = LogisticRegression()

# Fit the model on the training data
# rf_classifier.fit(X_train, y_train)
lr_classifier.fit(X, y)


LogisticRegression()

In [32]:
wk_40 = eggs[eggs["fisc_wk_id"] == 202340]

In [33]:
# cust_nbr_list = wk_40["cust_nbr"].tolist()

cust_nbr_list = customers.tolist()

In [34]:

week_202340_cust_nbr = pd.DataFrame({
    'year': [2023] * len(cust_nbr_list),  
    'week': [40] * len(cust_nbr_list), 
    'cust_nbr': cust_nbr_list 
})

# Make predictions for week 202340
predictions = rf_classifier.predict(week_202340_cust_nbr)


In [74]:
lr_predictions = lr_classifier.predict(week_202340_cust_nbr)

In [76]:
rf_classifier.predict_proba(week_202340_cust_nbr)[:8]

array([[0.7 , 0.3 ],
       [0.92, 0.08],
       [0.88, 0.12],
       [0.94, 0.06],
       [0.16, 0.84],
       [0.16, 0.84],
       [0.87, 0.13],
       [0.93, 0.07]])

In [55]:
predictions

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,

In [35]:
len(predictions)

816

In [57]:
actual = pd.DataFrame({"fisc_wk_id1":[202340]*len(cust_nbr_list),"cust_nbr1" :cust_nbr_list})

In [58]:
actual

,fisc_wk_id1,cust_nbr1
0,202340,1628
1,202340,3129
2,202340,10470
3,202340,11759
4,202340,11846
...,...,...
811,202340,35014
812,202340,16491
813,202340,35487
814,202340,19362


In [59]:
actual = actual.merge(wk_40[["fisc_wk_id","cust_nbr"]], 
             left_on = ["fisc_wk_id1","cust_nbr1"] ,
             right_on = ["fisc_wk_id","cust_nbr"], 
             how = "left")

In [61]:
actual

,fisc_wk_id1,cust_nbr1,fisc_wk_id,cust_nbr
0,202340,1628,NaN,NaN
1,202340,3129,NaN,NaN
2,202340,10470,NaN,NaN
3,202340,11759,NaN,NaN
4,202340,11846,202340.0,11846.0
...,...,...,...,...
811,202340,35014,NaN,NaN
812,202340,16491,NaN,NaN
813,202340,35487,NaN,NaN
814,202340,19362,NaN,NaN


In [62]:
actual["actual_purchases"] = 0

In [63]:
actual.loc[actual["fisc_wk_id"].notnull(), "actual_purchases"] = 1

In [64]:
actual

,fisc_wk_id1,cust_nbr1,fisc_wk_id,cust_nbr,actual_purchases
0,202340,1628,NaN,NaN,0
1,202340,3129,NaN,NaN,0
2,202340,10470,NaN,NaN,0
3,202340,11759,NaN,NaN,0
4,202340,11846,202340.0,11846.0,1
...,...,...,...,...,...
811,202340,35014,NaN,NaN,0
812,202340,16491,NaN,NaN,0
813,202340,35487,NaN,NaN,0
814,202340,19362,NaN,NaN,0


In [65]:
actual["predictions"] = predictions

In [77]:
actual["lr_predictions"] = lr_predictions

In [78]:
actual.columns

Index(['fisc_wk_id1', 'cust_nbr1', 'fisc_wk_id', 'cust_nbr',
       'actual_purchases', 'predictions', 'lr_predictions'],
      dtype='object')

In [79]:
actual_predict = actual[['fisc_wk_id1', 'cust_nbr1',
       'actual_purchases', 'predictions', 'lr_predictions']]

In [80]:
actual

,fisc_wk_id1,cust_nbr1,fisc_wk_id,cust_nbr,actual_purchases,predictions,lr_predictions
0,202340,1628,NaN,NaN,0,0,0
1,202340,3129,NaN,NaN,0,0,0
2,202340,10470,NaN,NaN,0,0,0
3,202340,11759,NaN,NaN,0,0,0
4,202340,11846,202340.0,11846.0,1,1,0
...,...,...,...,...,...,...,...
811,202340,35014,NaN,NaN,0,0,0
812,202340,16491,NaN,NaN,0,0,0
813,202340,35487,NaN,NaN,0,0,0
814,202340,19362,NaN,NaN,0,0,0


In [81]:
actual_predict["matches"] = actual_predict["actual_purchases"] == actual_predict["predictions"]

C:\Users\Waqas\AppData\Local\Temp\ipykernel_18172\1366138699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_predict["matches"] = actual_predict["actual_purchases"] == actual_predict["predictions"]


In [84]:
actual_predict["lr_matches"] = actual_predict["actual_purchases"] == actual_predict["lr_predictions"]

In [85]:
actual_predict

,fisc_wk_id1,cust_nbr1,actual_purchases,predictions,lr_predictions,matches,lr_matches
0,202340,1628,0,0,0,True,True
1,202340,3129,0,0,0,True,True
2,202340,10470,0,0,0,True,True
3,202340,11759,0,0,0,True,True
4,202340,11846,1,1,0,True,False
...,...,...,...,...,...,...,...
811,202340,35014,0,0,0,True,True
812,202340,16491,0,0,0,True,True
813,202340,35487,0,0,0,True,True
814,202340,19362,0,0,0,True,True


In [83]:
actual_predict.to_csv("complete_report_202340.csv", index = False)

In [46]:
accuracy = actual_predict["matches"].sum()/len(actual_predict)

In [47]:
accuracy

0.8394607843137255

In [86]:
lr_accuracy = actual_predict["lr_matches"].sum()/len(actual_predict)
lr_accuracy

0.7696078431372549

In [48]:
# customer who will expect to buy in Fisc_wk_id = 202340 according to their purchasing pattern.

expect = actual_predict[actual_predict["predictions"]== 1]["cust_nbr1"]

In [49]:
expect.to_csv("expect_buy_202340.csv", index = False)

In [50]:
# customer who actually buy in Fisc_wk_id = 202340

actually_buy = actual_predict[actual_predict["actual_purchases"]== 1]["cust_nbr1"]

In [51]:
actually_buy.to_csv("actual_buy_202340.csv", index = False)

In [52]:

churned = actual[(actual_predict["actual_purchases"]== 0) & (actual_predict["predictions"]== 1)]["cust_nbr1"]

In [53]:
churned.to_csv("customer_churned_202340.csv", index = False)